# Dev-work: hashing as parsing

In [3]:
# system dependecies
import os
from pathlib import Path
import time

# library dependencies
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import seaborn as sns
from collections import defaultdict

## biopython
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SearchIO

## pyhmmer
import pyhmmer

## datasketch
from datasketch import MinHash

# local dependencies/utils

## Paths
PFAM_PATH = Path("/Users/humoodalanzi/pfam/Pfam-A.hmm")
ID_DB_PATH = Path("/Users/humoodalanzi/pfam/proteins_id.zip")
#probably need path of unit tests

In [3]:
meso_output = os.path.abspath(os.path.join('..', 'examples', "meso_output.domtblout"))
thermo_output = os.path.abspath(os.path.join('..', 'examples', "thermo_output.domtblout"))